In [1]:
import pickle
import joblib
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import csv

In [2]:
def openStopword():
    f = open('dataset/stopwords/stopwords.csv', 'r', encoding='utf-8')
    reader = csv.reader(f)
    stopwords = list()

    for row in reader:
        stopwords.append(row[0])

    return stopwords

In [3]:
def tokenizer(raw, pos=["Noun","Verb"], stopword=openStopword()):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,
            stem=True
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [4]:
okt = Okt()

In [5]:
keyword_names = ['글로벌역량', '능동', '도전', '성실', '소통', '인내심', '정직', '주인의식', '창의', '팀워크']

In [6]:
vectorize = TfidfVectorizer(
    ngram_range=(1,3),
    tokenizer=tokenizer,
    max_df=0.95,
    min_df=0,
    sublinear_tf=True
)

In [7]:
filename = 'saved_model/SVC_190516.joblib'
svc_from_pickle = joblib.load(filename)

In [8]:
def SVCpredict(model,text):
    result = model.predict([text])
    return keyword_names[result[0]-1]

In [9]:
import json
#기업
#삼성 현대 엘지 SK CJ

samsung_origin = json.loads(open('dataset/resumes/educe_samsung.json', encoding="utf-8").read())
hyundai_origin = json.loads(open('dataset/resumes/educe_hyundai.json', encoding="utf-8").read())
LG_origin = json.loads(open('dataset/resumes/educe_LG.json', encoding="utf-8").read())
SK_origin = json.loads(open('dataset/resumes/educe_SK.json', encoding="utf-8").read())
CJ_origin = json.loads(open('dataset/resumes/educe_CJ.json', encoding="utf-8").read())

In [10]:
def make_txt(origin_json):
    result = ""
    for resume in origin_json:
        result += resume['document']
    return result

In [11]:
samsung = make_txt(samsung_origin)
hyundai = make_txt(hyundai_origin)
LG = make_txt(LG_origin)
SK = make_txt(SK_origin)
CJ = make_txt(CJ_origin)

In [12]:
predict_result = svc_from_pickle.predict([samsung,hyundai,LG,SK,CJ]).tolist()

In [13]:
decision_result = svc_from_pickle.decision_function([samsung,hyundai,LG,SK,CJ]).tolist()

In [14]:
with open('predict_190517.txt', 'w') as f:
    for item in predict_result:
        f.write("%s\n" % item)

In [15]:
with open('decision_190517.txt', 'w') as f:
    for item in decision_result:
        f.write("%s\n" % item)